In [1]:
import os
import requests
import json

from bdrk.v1 import ApiClient, Configuration, PipelineApi
from bdrk.v1.models import (
    PipelineResourcesSchema,
    TrainingPipelineRunSchema,
    GitSourceSchema
)

In [2]:
# Params for Bedrock Client
# Not required if training pipeline is run via UI
PROJECT_ID = "victor-sandbox"
PIPELINE_ID = "credit-scoring-delta"
ENVIRONMENT_ID = "sandbox-aws-production"

### Helper functions

In [3]:
def fetch_pipeline_default_values(bedrock_access_token, project_id, pipeline_id):
    '''
    Calls the Bedrock REST API to fetch pipeline default values
    '''
    
    url = "https://api.bdrk.ai/v1/training_pipeline/{pipeline_id}/default_values/".format(pipeline_id=pipeline_id)
    headers = {
        'X-Bedrock-Access-Token': bedrock_access_token,
        'Content-Type': 'application/json',
    }
    params = {'project_id': project_id}
    r = requests.get(url, headers=headers, params=params)
    response_json = json.loads(r.text)

    return response_json

## Step 1: Configure the Bedrock Client Library

In [4]:
configuration = Configuration()
configuration.api_key["X-Bedrock-Access-Token"] = os.environ["BEDROCK_API_TOKEN"]
configuration.host = "https://api.bdrk.ai"

In [5]:
# Fetch pipeline default values
pipeline_default_values = fetch_pipeline_default_values(
    bedrock_access_token=os.environ["BEDROCK_API_TOKEN"], 
    project_id=PROJECT_ID, 
    pipeline_id=PIPELINE_ID
)

In [6]:
# Init the client
api_client = ApiClient(configuration)
pipeline_api = PipelineApi(api_client)

# Example: change the commit hash of TrainingPipelineRun
# Replace value with the string commit hash
source = {
    "commit": pipeline_default_values["id"],
}

# Example: change Bedrock HCL environment parameters at run-time
script_parameters= {
    "SEED": "4"
}

# Configure the run
run_schema = TrainingPipelineRunSchema(
    environment_id=ENVIRONMENT_ID,
    source=source,
    script_parameters=script_parameters
)

## Step 2: Run the pipeline

In [7]:
# Submit the request to run
run = pipeline_api.run_training_pipeline(
    project_id=PROJECT_ID,
    pipeline_id=PIPELINE_ID, 
    training_pipeline_run_schema=run_schema
)